In [1]:
import os
os.getcwd()

'C:\\Users\\abhishekreddy\\Desktop\\TAMU\\Sem 2\\STAT - 656\\HW5 Solutions'

In [2]:
# Use Advanced Analytics for printing results and encoding
from AdvancedAnalytics import ReplaceImputeEncode, NeuralNetwork

# sklearn classes for fitting and evaluating classification networks
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

# other needed classes for converting data and results
import pandas as pd
import numpy as np

In [4]:
df = pd.read_excel("CreditHistory_Clean.xlsx")

In [6]:
# First Value Designates Data Type
# 'I'=Interval, 'B'=Binary, 'N'=Nominal, 'Z'=Other (No Changes, Exclude)
attribute_map = { \
            'age':['I',(19,120)], \
            'amount': ['I',(0,20000)], \
            'checking': ['N',(1,2,3,4)], \
            'coapp': ['N',(1,2,3)], \
            'depends': ['B',(1,2)], \
            'duration': ['I',(1,72)], \
            'employed': ['N',(1,2,3,4,5)], \
            'existcr': ['N',(1,2,3,4)], \
            'foreign': ['B',(1,2)], \
            'good_bad': ['B',('bad','good')], \
            'history': ['N',(0,1,2,3,4)], \
            'housing':['N',(1,2,3)], \
            'installp': ['N',(1,2,3,4)], \
            'job': ['N',(1,2,3,4)], \
            'marital': ['N',(1,2,3,4)], \
            'other': ['N',(1,2,3)], \
            'property': ['N',(1,2,3,4)], \
            'purpose': ['N',('0', '1','2','3','4','5','6', '8','9','X')],\
            'resident': ['N',(1,2,3,4)], \
            'savings': ['N',(1,2,3,4,5)], \
            'telephon': ['B',(1,2)] \
             }

In [7]:
'''
We use the class ReplaceImputeEncode() to identify outliers, replace them with missing values and then impute those values. 
In this case, the data are clean. This is not necessary, but we do need one-hot encoding for the nominal
attributes, and we do not want to drop the last of these encoded columns. 
For this we set nominal encoding to one-hot and drop to False.
'''

# drop=False - do not drop last category - used for Decision Trees
rie = ReplaceImputeEncode(data_map = attribute_map, nominal_encoding = 'one-hot', \
                          interval_scale = None, drop = False, display = True)

#features_map = rie.draft_features_map(df)
encoded_df = rie.fit_transform(df)


********** Data Preprocessing ***********
Features Dictionary Contains:
3 Interval, 
4 Binary, 
14 Nominal, and 
0 Excluded Attribute(s).

Data contains 1000 observations & 21 columns.


Attribute Counts
............... Missing  Outliers
age.......         0         0
amount....         0         0
checking..         0         0
coapp.....         0         0
depends...         0         0
duration..         0         0
employed..         0         0
existcr...         0         0
foreign...         0         0
good_bad..         0         0
history...         0         0
housing...         0         0
installp..         0         0
job.......         0         0
marital...         0         0
other.....         0         0
property..         0         0
purpose...         0         0
resident..         0         0
savings...         0         0
telephon..         0         0


In [9]:
y = encoded_df['good_bad'] # The target is not scaled or imputed
X = encoded_df.drop('good_bad',axis=1)
# convert pandas column vector 'y' to a numpy row vector 'np_y'
np_y = np.ravel(y)

In [10]:
# Cross-Validation
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html
# https://datascience.stackexchange.com/questions/28441/cross-validate-and-cross-val-score-in-scikitlearn
best_network = 0
best_alpha = 0
best_F1 = 0
network_list = [(3), (11), (5,4), (6,5), (7,6), (8,7)]
alpha_list = [0, 0.1, 0.15, 0.2, 0.25, 0.3]
score_list = ['accuracy', 'recall', 'precision', 'f1']

for nn in network_list:
    print("\nNetwork: ", nn)
    for a in alpha_list:
        print("\nAlpha: ", a)
        fnn = MLPClassifier(hidden_layer_sizes=nn, activation='tanh', alpha=a, tol=1e-32, \
                            solver='lbfgs', max_iter=5000, random_state=12345)
        scores = cross_validate(fnn, X, np_y, scoring=score_list, return_train_score=False, cv=10)
        print("{:.<13s}{:>6s}{:>13s}".format("Metric", "Mean", "Std. Dev."))
        for s in score_list:
            var = "test_"+s
            mean = scores[var].mean()
            std = scores[var].std()
            print("{:.<13s}{:>7.4f}{:>10.4f}".format(s, mean, std))
            if s == "f1" and best_F1 < mean:
                best_F1 = mean
                best_network = nn
                best_alpha = a


Network:  3

Alpha:  0
Metric.......  Mean    Std. Dev.
accuracy..... 0.7000    0.0000
recall....... 1.0000    0.0000
precision.... 0.7000    0.0000
f1........... 0.8235    0.0000

Alpha:  0.1
Metric.......  Mean    Std. Dev.
accuracy..... 0.7460    0.0418
recall....... 0.8914    0.0674
precision.... 0.7844    0.0516
f1........... 0.8310    0.0247

Alpha:  0.15
Metric.......  Mean    Std. Dev.
accuracy..... 0.7360    0.0329
recall....... 0.8329    0.0367
precision.... 0.7999    0.0307
f1........... 0.8153    0.0229

Alpha:  0.2
Metric.......  Mean    Std. Dev.
accuracy..... 0.7420    0.0382
recall....... 0.8457    0.0469
precision.... 0.7978    0.0218
f1........... 0.8206    0.0293

Alpha:  0.25
Metric.......  Mean    Std. Dev.
accuracy..... 0.7420    0.0384
recall....... 0.8400    0.0355
precision.... 0.8021    0.0324
f1........... 0.8201    0.0263

Alpha:  0.3
Metric.......  Mean    Std. Dev.
accuracy..... 0.7340    0.0432
recall....... 0.8343    0.0448
precision.... 0.7964    0.032

In [11]:
'''
Although the differences in F1 between the networks is small, the (7, 6) network has the highest 
F1 score with regularization set by alpha=0.3. This network will be selected as the best.
'''
print("The Best Network with F1=", best_F1)
print(" Perceptrons: ", best_network)
print(" Alpha: ", best_alpha)

The Best Network with F1= 0.8310156087629903
 Perceptrons:  3
 Alpha:  0.1


In [12]:
# Evaluate the tree with the best depth
X_train, X_validate, y_train, y_validate = train_test_split(X, np_y,test_size = 0.3, random_state=7)
fnn = MLPClassifier(hidden_layer_sizes=best_network, activation='tanh', \
                    alpha=best_alpha, tol=1e-32, solver='lbfgs', \
                    max_iter=5000, random_state=12345)
fnn = fnn.fit(X_train,y_train)
NeuralNetwork.display_binary_split_metrics(fnn, X_train, y_train, X_validate, y_validate)



Model Metrics..........       Training     Validation
Observations...........            700            300
Features...............             68             68
Number of Layers.......              1              1
Number of Outputs......              1              1
Number of Neurons......              3              3
Number of Weights......            211            211
Number of Iterations...           2180           2180
Activation Function....       logistic       logistic
Mean Absolute Error....         0.2839         0.2945
Avg Squared Error......         0.1432         0.1621
Accuracy...............         0.7957         0.7400
Precision..............         0.8542         0.8357
Recall (Sensitivity)...         0.8489         0.7972
F1-score...............         0.8515         0.8160
MISC (Misclassification)...      20.4%          26.0%
     class 0...............      32.3%          41.0%
     class 1...............      15.1%          20.3%


Training
Confusion Matri